
При помощи Faker и numpy генерируются данные.

Создается 4 таблицы:
- 1000 физ.лиц/ИП
- 1000 юр.лиц
- для каждого физ.лица/ИП/юр.лица создается по 1 счету - итого 2000 счетов
- для каждого счета создается по 10 операций во временном отрезке 60 дней, итого 20000 транзакций.

Файлы создаются по пути C:\datasets\

In [ ]:
#!pip install faker

In [ ]:
import pandas as pd
import random
import numpy as np
from faker import Faker
from datetime import datetime, timedelta

In [ ]:
# Инициализация Faker
fake = Faker('ru_RU')

## Генерация Client_FL (Физические лица)

In [ ]:
# Количество строк данных
n_samples = 1000

In [ ]:
# Списки возможных значений
client_types = ['ИП', 'ФЛ']
genders = ['М', 'Ж']
citizenships = ['RUS', 'BLR', 'KZ', 'TJ', 'KG', 'AZ']
tax_statuses = ['Y', 'N']
risk_levels = ['НИЗКИЙ', 'СРЕДНИЙ', 'ВЫСОКИЙ']
marital_statuses = ['ХОЛОСТ', 'ЖЕНАТ/ЗАМУЖЕМ', 'РАЗВЕДЕН', 'ВДОВЕЦ/ВДОВА']
employment_statuses = [
    'НАЕМНЫЙ РАБОТНИК',
    'ИНДИВИДУАЛЬНЫЙ ПРЕДПРИНИМАТЕЛЬ',
    'БЕЗРАБОТНЫЙ',
    'САМОЗАНЯТЫЙ',
    'ГОСУДАРСТВЕННЫЙ СЛУЖАЩИЙ',
    'СТУДЕНТ',
    'ПЕНСИОНЕР',
    'ВОЕННОСЛУЖАЩИЙ',
    'ГЛАВА КФХ'
]
pep_statuses = ['Y', 'N']

In [ ]:
# Функция для генерации случайных дат
#def random_date(start_date, end_date):
#    return start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

# Функция для генерации случайных дат
def random_date(start_date, end_date):
    return start_date + (end_date - start_date) * random.random()

# Текущая дата для генерации дат
current_date = datetime.now()

In [ ]:
# Генерация данных ФЛ и ИП
data = []
for _ in range(n_samples):
    client_type = random.choice(client_types)
    gender = random.choice(genders)
    citizenship = random.choice(citizenships)
    tax_status = random.choice(tax_statuses)
    risk_level = random.choice(risk_levels)
    marital_status = random.choice(marital_statuses)
    employment_status = random.choice(employment_statuses)
    pep_status = random.choice(pep_statuses)

    client_id = f"{random.randint(10000, 99999)}_{fake.bothify(text='???').upper()}"
    if gender == 'М':
        full_name = f"{fake.last_name_male()} {fake.first_name_male()} {fake.middle_name_male()}"
    else:
        full_name = f"{fake.last_name_female()} {fake.first_name_female()} {fake.middle_name_female()}"
    inn = ''.join([str(random.randint(0, 9)) for _ in range(12)])
    registration_number = np.nan if client_type == 'ФЛ' else ''.join([str(random.randint(0, 9)) for _ in range(15)])
    registration_date = np.nan if registration_number is np.nan else random_date(datetime(2000, 1, 1), current_date).strftime('%Y-%m-%d')
    okved_code = np.nan if registration_number is np.nan else f"{random.randint(10, 99)}.{random.randint(10, 99)}"
    date_of_birth = fake.date_of_birth(minimum_age=18, maximum_age=80).strftime('%Y-%m-%d')
    place_of_birth = fake.city()
    document_type = random.choice(['ПАСПОРТ', 'ИНОСТРАННЫЙ ПАСПОРТ'])
    document_number = fake.unique.numerify(text='###########')
    document_issue_date = fake.date_between(start_date='-10y', end_date='-1d').strftime('%Y-%m-%d')
    document_expiry_date = random_date(datetime(2025, 1, 1), datetime(2035, 12, 31)).strftime('%Y-%m-%d')
    document_issuing_authority = fake.company()
    residential_address = fake.address().replace('\n', ', ')
    registration_address = np.nan if random.random() > 0.5 else fake.address().replace('\n', ', ')
    postal_address = np.nan if random.random() > 0.5 else fake.address().replace('\n', ', ')
    contact_phone = fake.phone_number()
    last_kyc_review_date = random_date(datetime(2023, 1, 1), current_date).strftime('%Y-%m-%d')
    created_at = datetime.combine(fake.date_this_year(), datetime.min.time()).strftime('%Y-%m-%dT%H:%M:%S')
    updated_at = created_at  # Для простоты считаем, что обновление происходит в момент создания записи

    # Собираем все данные в одну строку
    data.append([
        client_id, full_name, client_type, inn, registration_number, registration_date, okved_code,
        date_of_birth, place_of_birth, gender, citizenship, tax_status, citizenship, document_type,
        document_number, document_issue_date, document_expiry_date, document_issuing_authority,
        residential_address, registration_address, postal_address, contact_phone, marital_status,
        employment_status, pep_status, risk_level, last_kyc_review_date, created_at, updated_at
    ])

# Создание DataFrame
columns = [
    'client_id', 'full_name', 'client_type', 'inn', 'registration_number', 'registration_date', 'okved_code',
    'date_of_birth', 'place_of_birth', 'gender', 'citizenship', 'tax_registration_status', 'tax_registration_country',
    'document_type', 'document_number', 'document_issue_date', 'document_expiry_date', 'document_issuing_authority',
    'residential_address', 'registration_address', 'postal_address', 'contact_phone', 'marital_status',
    'employment_status', 'pep_status', 'risk_level', 'last_kyc_review_date', 'created_at', 'updated_at'
]

client_fl = pd.DataFrame(data, columns=columns)

client_fl.head()

,client_id,full_name,client_type,inn,registration_number,registration_date,okved_code,date_of_birth,place_of_birth,gender,...,registration_address,postal_address,contact_phone,marital_status,employment_status,pep_status,risk_level,last_kyc_review_date,created_at,updated_at
0,76506_FNN,Белова Юлия Артемовна,ИП,913918011098,939614485029819,2018-04-19,48.30,1988-02-11,г. Белокуриха,Ж,...,"г. Обоянь, ш. Высоковольтное, д. 52, 336804","клх Ноглики, ш. Пролетарское, д. 364, 312642",86444928541,ВДОВЕЦ/ВДОВА,ГЛАВА КФХ,Y,СРЕДНИЙ,2024-01-16,2024-01-16T00:00:00,2024-01-16T00:00:00
1,57114_ENR,Воронцов Варлаам Фёдорович,ФЛ,552607984417,NaN,NaN,NaN,1944-07-12,г. Котлас,М,...,"к. Шилка, пер. М.Горького, д. 737, 149937","г. Ржев, наб. Чехова, д. 387 к. 3/6, 614177",8 470 051 9130,ХОЛОСТ,ИНДИВИДУАЛЬНЫЙ ПРЕДПРИНИМАТЕЛЬ,N,НИЗКИЙ,2023-05-10,2024-09-05T00:00:00,2024-09-05T00:00:00
2,78379_SDL,Сорокин Эрнст Адамович,ФЛ,407790894899,NaN,NaN,NaN,1975-12-08,к. Данков,М,...,"п. Ялуторовск, ул. Свободная, д. 5 к. 959, 009610",NaN,8 643 221 8499,ХОЛОСТ,БЕЗРАБОТНЫЙ,N,СРЕДНИЙ,2023-11-03,2024-01-17T00:00:00,2024-01-17T00:00:00
3,92644_UFI,Крюкова Анна Владиславовна,ИП,374623900372,424188642293372,2001-01-24,50.39,1988-02-13,к. Невьянск,Ж,...,NaN,NaN,8 169 584 5405,РАЗВЕДЕН,ГОСУДАРСТВЕННЫЙ СЛУЖАЩИЙ,N,СРЕДНИЙ,2023-06-15,2024-09-15T00:00:00,2024-09-15T00:00:00
4,72529_PXX,Ларионов Любим Давидович,ФЛ,036260052161,NaN,NaN,NaN,1989-02-17,с. Казань,М,...,"с. Любань, ул. Олимпийская, д. 7 стр. 9/4, 845785","д. Горячинск, ш. Профсоюзное, д. 72, 737557",+7 992 245 62 98,ВДОВЕЦ/ВДОВА,ГОСУДАРСТВЕННЫЙ СЛУЖАЩИЙ,N,ВЫСОКИЙ,2024-08-22,2024-07-08T00:00:00,2024-07-08T00:00:00


## Генерация Client_UL (Юридические лица)

In [ ]:
# Количество строк данных
n_samples = 1000

In [ ]:
# Коды ОКВЭД букмекерских ЮЛ

okved_gambling = ['92.11', '92.12', '92.13', '92.21', '92.22', '92.23']

# Остальные коды ОКВЭД

okved_regular = [
    '01.11', '01.12', '01.13', '01.14', '01.15', '01.16', '01.19',
    '01.21', '01.22', '01.23', '01.24', '01.25', '01.26', '01.27', '01.28', '01.29',
    '01.30', '01.41', '01.42', '01.43', '01.44', '01.45', '01.46', '01.47', '01.49',
    '01.50', '01.61', '01.62', '01.63', '01.64', '01.70',
    '02.10', '02.20', '02.30', '02.40',
    '03.11', '03.12', '03.21', '03.22',
    '05.10', '05.20',
    '06.10', '06.20',
    '07.10', '07.21', '07.29',
    '08.11', '08.12', '08.91', '08.92', '08.93', '08.99',
    '09.10', '09.90',
    '10.11', '10.12', '10.13', '10.20', '10.31', '10.32', '10.39', '10.41', '10.42', '10.51', '10.52', '10.61', '10.62', '10.71', '10.72', '10.73', '10.81', '10.82', '10.83', '10.84', '10.85', '10.86', '10.89', '10.91', '10.92',
    '11.01', '11.02', '11.03', '11.04', '11.05', '11.06', '11.07',
    '12.00',
    '13.10', '13.20', '13.30', '13.91', '13.92', '13.93', '13.94', '13.95', '13.96', '13.99',
    '14.11', '14.12', '14.13', '14.14', '14.19', '14.20', '14.31', '14.39',
    '15.11', '15.12', '15.20',
    '16.10', '16.21', '16.22', '16.23', '16.24', '16.29',
    '17.11', '17.12', '17.21', '17.22', '17.23', '17.24', '17.29',
    '18.11', '18.12', '18.13', '18.14', '18.20',
    '19.10', '19.20',
    '20.11', '20.12', '20.13', '20.14', '20.15', '20.16', '20.17', '20.20', '20.30', '20.41', '20.42', '20.51', '20.52', '20.53', '20.59',
    '21.10', '21.20',
    '22.11', '22.19', '22.21', '22.22', '22.23', '22.29',
    '23.11', '23.12', '23.13', '23.14', '23.19', '23.20', '23.31', '23.32', '23.41', '23.42', '23.43', '23.44', '23.49', '23.51', '23.52', '23.61', '23.62', '23.63', '23.64', '23.65', '23.69', '23.70', '23.91', '23.99',
    '24.10', '24.20', '24.31', '24.32', '24.33', '24.34', '24.39', '24.41', '24.42', '24.43', '24.44', '24.45', '24.46', '24.51', '24.52', '24.53', '24.54',
    '25.11', '25.12', '25.21', '25.29', '25.30', '25.40', '25.50', '25.61', '25.62', '25.71', '25.72', '25.73', '25.91', '25.92', '25.93', '25.94', '25.99',
    '26.11', '26.12', '26.20', '26.30', '26.40', '26.51', '26.52', '26.60', '26.70', '26.80',
    '27.11', '27.12', '27.20', '27.31', '27.32', '27.33', '27.40', '27.51', '27.52', '27.90',
    '28.11', '28.12', '28.13', '28.14', '28.15', '28.16', '28.17', '28.18', '28.19', '28.21', '28.22', '28.23', '28.24', '28.25', '28.29', '28.30', '28.41', '28.49', '28.91', '28.92', '28.93', '28.94', '28.95', '28.96', '28.99',
    '29.10', '29.20', '29.31', '29.32',
    '30.11', '30.12', '30.20', '30.30', '30.40', '30.91', '30.92', '30.99',
    '31.01', '31.02', '31.03', '31.09',
    '32.11', '32.12', '32.13', '32.20', '32.30', '32.40', '32.50', '32.91', '32.99',
    '33.11', '33.12', '33.13', '33.14', '33.15', '33.16', '33.17', '33.19', '33.20',
    '35.11', '35.12', '35.13', '35.14', '35.21', '35.22', '35.23', '35.30',
    '36.00',
    '37.00',
    '38.11', '38.12', '38.21', '38.22', '38.31', '38.32',
    '39.00',
    '41.10', '41.20',
    '42.11', '42.12', '42.13', '42.21', '42.22', '42.91', '42.99',
    '43.11', '43.12', '43.13', '43.21', '43.22', '43.29', '43.31', '43.32', '43.33', '43.34', '43.39', '43.91', '43.99',
    '45.11', '45.19', '45.20', '45.31', '45.32', '45.40',
    '46.11', '46.12', '46.13', '46.14', '46.15', '46.16', '46.17', '46.18', '46.19', '46.21', '46.22', '46.23', '46.24', '46.31', '46.32', '46.33', '46.34', '46.35', '46.36', '46.37', '46.38', '46.39', '46.41', '46.42', '46.43', '46.44', '46.45', '46.46', '46.47', '46.48', '46.49', '46.51', '46.52', '46.61', '46.62', '46.63', '46.64', '46.65', '46.66', '46.69', '46.71', '46.72', '46.73', '46.74', '46.75', '46.76', '46.77', '46.90',
    '47.11', '47.19', '47.21', '47.22', '47.23', '47.24', '47.25', '47.26', '47.29', '47.30', '47.41', '47.42', '47.43', '47.51', '47.52', '47.53', '47.54', '47.59', '47.61', '47.62', '47.63', '47.64', '47.65', '47.71', '47.72', '47.73', '47.74', '47.75', '47.76', '47.77', '47.78', '47.79', '47.81', '47.82', '47.89', '47.91', '47.99',
    '49.10', '49.20', '49.31', '49.32', '49.39', '49.41', '49.42', '49.50',
    '50.10', '50.20', '50.30', '50.40',
    '51.10', '51.21', '51.22',
    '52.10', '52.21', '52.22', '52.23', '52.24', '52.29',
    '53.10', '53.20',
    '55.10', '55.20', '55.30', '55.90',
    '56.10', '56.21', '56.29', '56.30',
    '58.11', '58.12', '58.13', '58.14', '58.19', '58.21', '58.29',
    '59.11', '59.12', '59.13', '59.14', '59.20',
    '60.10', '60.20',
    '61.10', '61.20', '61.30', '61.90',
    '62.01', '62.02', '62.03', '62.09',
    '63.11', '63.12', '63.91', '63.99',
    '64.11', '64.19', '64.20', '64.30', '64.91', '64.92', '64.99',
    '65.11', '65.12', '65.20', '65.30',
    '66.11', '66.12', '66.19', '66.21', '66.22', '66.29', '66.30',
    '68.10', '68.20', '68.31', '68.32',
    '69.10', '69.20',
    '70.10', '70.21', '70.22',
    '71.11', '71.12', '71.20',
    '72.11', '72.19', '72.20',
    '73.11', '73.12', '73.20',
    '74.10', '74.20', '74.30', '74.90',
    '75.00',
    '77.11', '77.12', '77.21', '77.22', '77.29', '77.31', '77.32', '77.33', '77.34', '77.35', '77.39', '77.40', '77.91', '77.92', '77.99',
    '78.10', '78.20', '78.30',
    '79.11', '79.12', '79.90',
    '80.10', '80.20', '80.30',
    '81.10', '81.21', '81.22', '81.29', '81.30',
    '82.11', '82.19', '82.20', '82.30', '82.91', '82.92', '82.99',
    '84.11', '84.12', '84.13', '84.21', '84.22', '84.23', '84.24', '84.25', '84.30',
    '85.10', '85.20', '85.31', '85.32', '85.41', '85.42', '85.51', '85.52', '85.53', '85.59', '85.60',
    '86.10', '86.21', '86.22', '86.23', '86.90',
    '87.10', '87.20', '87.30', '87.90',
    '88.10', '88.91', '88.99',
    '90.01', '90.02', '90.03', '90.04',
    '91.01', '91.02', '91.03', '91.04',
    '93.11', '93.12', '93.13', '93.19', '93.21', '93.29',
    '94.11', '94.12', '94.20', '94.91', '94.92', '94.99',
    '95.11', '95.12', '95.21', '95.22', '95.23', '95.24', '95.25', '95.29',
    '96.01', '96.02', '96.03', '96.04', '96.09',
    '97.00',
    '98.10', '98.20',
    '99.00'
]

In [ ]:
# Списки возможных значений
company_types = ['ООО', 'АО', 'ПАО']
risk_levels = ['НИЗКИЙ', 'СРЕДНИЙ', 'ВЫСОКИЙ']
tax_statuses = ['Y', 'N']
pep_statuses = ['Y', 'N']
industries = ['IT', 'СТРОИТЕЛЬСТВО', 'ФИНАНСЫ', 'ТРАНСПОРТ', 'ТОРГОВЛЯ', 'ЗДРАВООХРАНЕНИЕ']
countries = ['RUS', 'USA', 'GBR', 'BLR', 'FRA']
company_statuses = ['АКТИВНА', 'ЛИКВИДИРОВАНА', 'В СТАДИИ ЛИКВИДАЦИИ']

# Генерация данных ЮЛ
data = []
for _ in range(n_samples):
    client_id = f"{random.randint(10000, 99999)}_{fake.bothify(text='???').upper()}"
    legal_form = random.choice(company_types)
    company_name = f"{legal_form} {fake.word().capitalize()}"
    inn = ''.join([str(random.randint(0, 9)) for _ in range(10)])
    registration_date = random_date(datetime(2000, 1, 1), datetime(2024, 12, 31)).strftime('%Y-%m-%d')
    onboarding_date = random_date(datetime(2020, 1, 1), datetime(2024, 12, 31)).strftime('%Y-%m-%d')
    registration_number = ''.join([str(random.randint(0, 9)) for _ in range(13)])
    tax_registration_status = random.choice(tax_statuses)
    tax_registration_country = random.choice(countries)
    company_address = fake.address().replace('\n', ', ')
    postal_address = np.nan if random.random() > 0.5 else fake.address().replace('\n', ', ')
    industry_sector = random.choice(industries)
    # Логика для выбора okved_code
    if random.random() <= 0.25:  # ~25% строк заполняем из списка okved_gambling
        okved_code = random.choice(okved_gambling)
    else:  # Остальные строки заполняем из списка okved_regular
        okved_code = random.choice(okved_regular)
    business_activity_description = fake.sentence(nb_words=10)
    authorized_capital = random.randint(1000, 100000)
    ceo_name = fake.name()
    ceo_id = np.nan if random.random() > 0.5 else f"{random.randint(10000, 99999)}_CEO"
    ceo_document_type = random.choice(['ПАСПОРТ', 'ИДЕНТИФИКАЦИОННАЯ КАРТА', 'ВОДИТЕЛЬСКОЕ УДОСТОВЕРЕНИЕ'])
    ceo_document_number = fake.unique.numerify(text='###########')
    ceo_document_issue_date = fake.date_between(start_date='-10y', end_date='-1d').strftime('%Y-%m-%d')
    ceo_document_expiry_date = random_date(datetime(2025, 1, 1), datetime(2035, 12, 31)).strftime('%Y-%m-%d')
    ceo_inn = ''.join([str(random.randint(0, 9)) for _ in range(12)])
    beneficiary_name = fake.name()
    beneficiary_id = np.nan if random.random() > 0.5 else f"{random.randint(10000, 99999)}_BEN"
    beneficiaty_inn = ''.join([str(random.randint(0, 9)) for _ in range(12)])
    company_status = random.choices(company_statuses, weights=[0.95, 0.025, 0.025], k=1)[0]
    pep_status = random.choice(pep_statuses)
    foreign_ownership_flag = random.choice(['Y', 'N'])
    foreign_ownership_percentage = np.nan if foreign_ownership_flag == 'N' else random.randint(1, 100)
    risk_level = random.choice(risk_levels)
    kyc_status = random.choice(['Y', 'N'])
    last_kyc_review_date = random_date(datetime(2023, 1, 1), datetime(2024, 12, 31)).strftime('%Y-%m-%d')
    last_audit_date = random_date(datetime(2023, 1, 1), datetime(2024, 12, 31)).strftime('%Y-%m-%d')
    created_at = random_date(datetime(2023, 1, 1), datetime(2024, 12, 31)).strftime('%Y-%m-%dT%H:%M:%S')
    updated_at = created_at  # Для простоты считаем, что обновление происходит в момент создания записи

    # Собираем все данные в одну строку
    data.append([
        client_id, company_name, inn, registration_date, onboarding_date, registration_number, legal_form,
        tax_registration_status, tax_registration_country, company_address, postal_address, industry_sector,
        okved_code, business_activity_description, authorized_capital, ceo_name, ceo_id, ceo_document_type,
        ceo_document_number, ceo_document_issue_date, ceo_document_expiry_date, ceo_inn, beneficiary_name,
        beneficiary_id, beneficiaty_inn, company_status, pep_status, foreign_ownership_flag,
        foreign_ownership_percentage, risk_level, kyc_status, last_kyc_review_date, last_audit_date, created_at, updated_at
    ])

# Создание DataFrame
columns = [
    'client_id', 'company_name', 'inn', 'registration_date', 'onboarding_date', 'registration_number', 'legal_form',
    'tax_registration_status', 'tax_registration_country', 'company_address', 'postal_address', 'industry_sector',
    'okved_code', 'business_activity_description', 'authorized_capital', 'ceo_name', 'ceo_id', 'ceo_document_type',
    'ceo_document_number', 'ceo_document_issue_date', 'ceo_document_expiry_date', 'ceo_inn', 'beneficiary_name',
    'beneficiary_id', 'beneficiaty_inn', 'company_status', 'pep_status', 'foreign_ownership_flag',
    'foreign_ownership_percentage', 'risk_level', 'kyc_status', 'last_kyc_review_date', 'last_audit_date', 'created_at', 'updated_at'
]

client_ul = pd.DataFrame(data, columns=columns)

# Выводим первые строки для проверки
client_ul.head()

,client_id,company_name,inn,registration_date,onboarding_date,registration_number,legal_form,tax_registration_status,tax_registration_country,company_address,...,company_status,pep_status,foreign_ownership_flag,foreign_ownership_percentage,risk_level,kyc_status,last_kyc_review_date,last_audit_date,created_at,updated_at
0,52648_WRA,ООО Мягкий,8385189479,2015-09-07,2022-02-04,6570048461153,ООО,N,FRA,"ст. Абинск, ш. Дзержинского, д. 23 стр. 1/2, 3...",...,АКТИВНА,N,Y,99.0,ВЫСОКИЙ,N,2023-05-26,2023-05-24,2024-04-27T03:16:59,2024-04-27T03:16:59
1,38228_CWI,АО Бок,0141751056,2014-04-03,2023-07-14,0885206848361,АО,N,USA,"с. Карабудахкент, наб. Шахтерская, д. 6 стр. 9...",...,ЛИКВИДИРОВАНА,Y,N,NaN,ВЫСОКИЙ,Y,2024-09-23,2024-07-28,2024-01-03T14:05:08,2024-01-03T14:05:08
2,79356_LPK,АО Заложить,1159675696,2022-12-07,2023-12-04,9115169833134,АО,N,BLR,"ст. Кулунда, алл. Заводская, д. 98 стр. 6/3, 3...",...,АКТИВНА,Y,Y,10.0,СРЕДНИЙ,Y,2023-09-13,2023-07-22,2024-03-05T01:23:32,2024-03-05T01:23:32
3,58725_VTF,ООО Пастух,1336288046,2020-06-29,2020-06-08,2383388307867,ООО,Y,USA,"п. Рязань, пр. Ульяновский, д. 7 к. 5, 674557",...,АКТИВНА,Y,N,NaN,СРЕДНИЙ,N,2024-02-21,2024-10-28,2024-05-02T00:26:07,2024-05-02T00:26:07
4,29257_NYQ,ПАО Цепочка,6860427606,2013-01-17,2021-03-02,8274386718525,ПАО,Y,BLR,"п. Южноуральск, алл. М.Горького, д. 2 стр. 1/5...",...,АКТИВНА,N,N,NaN,ВЫСОКИЙ,Y,2024-03-14,2023-12-06,2024-03-27T00:49:41,2024-03-27T00:49:41


In [ ]:
# Проверем, сколько букмекерских среди получившихся ЮЛ
len(client_ul[client_ul['okved_code'].isin(okved_gambling)])

238

In [ ]:
client_ul['company_status'].value_counts()

,count
company_status,
АКТИВНА,951
ЛИКВИДИРОВАНА,26
В СТАДИИ ЛИКВИДАЦИИ,23


## Генерация Account (Счета)

In [ ]:
# Получаем все client_id из таблиц ФЛ и ЮЛ
all_client_ids = client_fl['client_id'].tolist() + client_ul['client_id'].tolist()

In [ ]:
# Списки возможных значений
account_types = ['ТЕКУЩИЙ', 'ДЕПОЗИТНЫЙ']
account_currencies = ['RUB', 'USD', 'EUR']
account_statuses = ['АКТИВНЫЙ', 'ЗАМОРОЖЕН', 'ЗАКРЫТ', 'СПЯЩИЙ']

In [ ]:
# Генерация данных счетов для каждого client_id
data = []
for client_id in all_client_ids: # Для каждого client_id из списка all_client_ids создаем один счет
    account_id = f"{random.randint(100000, 999999)}_CRD"
    account_number = f"4070{random.randint(100000000000, 999999999999)}"
    currency = random.choice(account_currencies)  # Валюта счета
    account_product = None if random.random() > 0.3 else 'КАРТА'
    if currency != 'RUB':
        account_type = 'ВАЛЮТНЫЙ'
    if account_product == 'КАРТА':
        account_type = 'КАРТОЧНЫЙ'
    else:
        account_type = random.choice(account_types)
    opening_date = random_date(datetime(2010, 1, 1), datetime(2024, 12, 31)).strftime('%Y-%m-%d')  # Дата открытия счета
    status = random.choices(account_statuses, weights=[0.8, 0.07, 0.07, 0.06], k=1)[0]
    if status == 'ЗАКРЫТ':
        closing_date = random_date(datetime(2020, 1, 1), datetime(2024, 12, 31)).strftime('%Y-%m-%d')  # Дата закрытия
    else:
        closing_date = np.nan  # Если счет не закрыт, то нет даты закрытия
        created_at = random_date(datetime(2020, 1, 1), datetime(2024, 12, 31)).strftime('%Y-%m-%dT%H:%M:%S')  # Дата создания
    updated_at = random_date(datetime(2020, 1, 1), datetime(2024, 12, 31)).strftime('%Y-%m-%dT%H:%M:%S')  # Дата обновления

    # Собираем данные для строки
    data.append([account_id, client_id, account_number, account_type, currency, opening_date, closing_date,
                 status, account_product, created_at, updated_at])

# Создание DataFrame
columns = ['account_id', 'client_id', 'account_number', 'account_type', 'currency', 'opening_date', 'closing_date',
           'status', 'account_product', 'created_at', 'updated_at']

account = pd.DataFrame(data, columns=columns)

account.sample(10)

,account_id,client_id,account_number,account_type,currency,opening_date,closing_date,status,account_product,created_at,updated_at
815,801028_CRD,41325_NTW,4070627005324200,КАРТОЧНЫЙ,USD,2019-03-11,NaN,АКТИВНЫЙ,КАРТА,2024-06-29T01:50:40,2024-11-26T04:22:22
577,704694_CRD,34634_QCK,4070420296474358,КАРТОЧНЫЙ,RUB,2016-06-05,NaN,АКТИВНЫЙ,КАРТА,2020-06-16T21:34:48,2024-07-04T10:47:37
98,930419_CRD,22531_TKR,4070893596059993,ДЕПОЗИТНЫЙ,EUR,2013-05-08,NaN,АКТИВНЫЙ,None,2021-03-07T05:49:19,2023-08-09T13:08:26
508,968094_CRD,89323_HTR,4070633358533790,ДЕПОЗИТНЫЙ,USD,2012-08-31,NaN,ЗАМОРОЖЕН,None,2020-01-18T02:19:55,2024-12-05T10:12:48
1193,307559_CRD,19229_VHY,4070810248625014,ТЕКУЩИЙ,USD,2015-08-16,NaN,АКТИВНЫЙ,None,2020-05-28T00:21:08,2024-04-16T00:24:59
1279,419226_CRD,53780_QFO,4070906372052145,ДЕПОЗИТНЫЙ,EUR,2021-02-10,NaN,АКТИВНЫЙ,None,2022-01-24T21:19:17,2024-10-14T01:02:01
1958,234143_CRD,96289_QVP,4070135750815727,ДЕПОЗИТНЫЙ,EUR,2014-09-15,NaN,АКТИВНЫЙ,None,2022-06-20T13:34:47,2021-11-15T11:42:06
783,975920_CRD,87153_MOS,4070440890541944,КАРТОЧНЫЙ,EUR,2018-08-27,NaN,АКТИВНЫЙ,КАРТА,2023-06-24T12:20:06,2020-08-13T05:55:58
913,620330_CRD,71980_RVI,4070138757506512,ДЕПОЗИТНЫЙ,RUB,2013-03-02,NaN,АКТИВНЫЙ,None,2023-08-18T03:54:11,2020-12-28T04:34:28
1656,752922_CRD,32635_QCL,4070191703658920,ДЕПОЗИТНЫЙ,EUR,2015-07-10,NaN,АКТИВНЫЙ,None,2022-05-22T22:56:40,2024-11-30T00:10:29


In [ ]:
# Проверка кол-ва счетов в разных статусах
account['status'].value_counts()

,count
status,
АКТИВНЫЙ,1604
ЗАМОРОЖЕН,141
СПЯЩИЙ,132
ЗАКРЫТ,123


## Генерация Transaction (Операции)

In [ ]:
# Списки возможных значений
operation_types = ['ЗАЧИСЛЕНИЕ', 'СПИСАНИЕ']
operation_statuses = ['ЗАВЕРШЕНА', 'ОЖИДАЕТСЯ', 'ОТКЛОНЕНА']
operation_channels = ['БАНКОМАТ', 'ИНТЕРНЕТ-БАНК', 'МОБИЛЬНОЕ ПРИЛОЖЕНИЕ', 'POS-ТЕРМИНАЛ']

possible_purposes = [ # Пример возможных назначений платежа
    'оплата за коммунальные услуги',
    'перевод средств на карту',
    'погашение кредита',
    'покупка товаров в интернет-магазине',
    'перевод на счет',
    'оплата по счету',
    'оплата услуг',
    'оплата мобильной связи',
    'пожертвование на благотворительность',
    'арендная плата',
    'оплата обучения',
    'ставка на игру',
    'ставка в казино',
    'ставки на спорт',
    'оплата ставки',
    'ставка на матч',
    'погашение ставки',
    'пари в казино',
    'пари на спортивное событие',
    'перевод пари',
    'оплата пари',
    'погашение пари',
    'выигрыш от участия в пари',
    'ставка в тотализатор',
    'оплата участия в тотализаторе',
    'погашение ставки в тотализаторе',
    'тотализатор на футбольный матч',
    'победа в тотализаторе',
    'участие в тотализаторе',
    'перевод в казино',
    'оплата за игру в казино',
    'перевод средств в казино',
    'выигрыш в казино',
    'перевод на счет казино',
    'оплата входа в казино',
    'ставка на игровом автомате',
    'игра на игровом автомате',
    'оплата за игру на автомате',
    'выигрыш с игрового автомата',
    'перевод на игровой автомат',
    'погашение долга за игровой автомат',
    'ставка на рулетку',
    'игра в рулетку',
    'оплата игры в рулетку',
    'перевод ставки в рулетку',
    'перевод средств на рулетку',
    'карточный долг',
    'оплата карточного долга',
    'перевод на погашение карточного долга',
    'погашение долга по кредитной карте',
    'внесение платежа по карточному долгу',
    'оплата задолженности по карточному долгу',
    'оплата по договору',
    'погашение задолженности',
    'перевод средств на личные нужды',
    'оплата аренды',
    'погашение долга',
    'оплата по контракту',
    'внесение аванса',
    'перевод на покупку товаров',
    'оплата долговых обязательств',
    'перевод на текущие расходы',
    'пожертвование на нужды организации',
    'оплата долгов по договору',
    'внесение средств на проект',
    'оплата услуг по подписке',
    'перевод на обслуживание счета',
    'перевод на обслуживание кредита',
    'оплата по текущим обязательствам',
    'перевод средств на банковский депозит',
    'оплата налоговых обязательств',
    'перевод на расчеты с поставщиками'
]

In [ ]:
# Генерация данных для операций
data = []
for _, acc in account.iterrows():
    for _ in range(10):  # Генерируем по 10 транзакций для каждого счета
        operation_id = f"{random.randint(100001, 999999)}_{fake.bothify(text='???').upper()}"
        client_id = acc['client_id']
        account_id = acc['account_id']
        account_number = acc['account_number']
        operation_channel = random.choice(operation_channels)
        operation_type = random.choice(operation_types)
        # Генерация случайной даты и времени совершения операции
        random_days = random.randint(0, 61)  # Случайное количество дней
        random_seconds = random.randint(0, 86400)  # Случайное количество секунд в пределах одного дня (86400 секунд в сутках)
        operation_date = (current_date - timedelta(days=random_days)).replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(seconds=random_seconds)
        operation_amount = random.randint(1000, 100000)  # Сумма операции
        currency = acc['currency']  # Используем валюту из account
        transaction_fee = random.choice([None, random.randint(1, 500)])  # Комиссия, может быть None
        operation_status = random.choices(operation_statuses, weights=[0.9, 0.05, 0.05], k=1)[0]
        counterparty_account_number = fake.credit_card_number() if random.random() < 0.5 else None
        counterparty_bank = fake.company() if counterparty_account_number else None
        counterparty_bank_country_code = fake.country_code() if counterparty_account_number else None
        counterparty_name = fake.name() if counterparty_account_number else None
        counterparty_inn = fake.ssn() if counterparty_account_number else None
        payment_purpose = random.choice(possible_purposes)
        terminal_id = None if operation_channel not in ['БАНКОМАТ', 'POS-ТЕРМИНАЛ'] else (random.randint(10000, 99999))
        location = np.nan if operation_channel in ['ИНТЕРНЕТ-БАНК', 'МОБИЛЬНОЕ ПРИЛОЖЕНИЕ'] else (fake.city() if random.random() > 0.5 else fake.country())
        document_number = fake.random_number(digits=7)
        exchange_rate = 1 if currency == 'RUB' else (100 if currency == 'EUR' else (90 if currency == 'USD' else np.nan))
        is_reversal = 'Y' if random.random() < 0.1 else 'N'
        linked_operation_id = None  # Нет связанной операции
        created_at = (operation_date + timedelta(hours=5)).strftime('%Y-%m-%dT%H:%M:%S')
        updated_at = created_at  # Обновление происходит в тот же день, что и создание

        # Собираем все данные в одну строку
        data.append([
            operation_id, client_id, account_id, account_number, operation_type, operation_date,
            operation_amount, currency, transaction_fee, operation_status, counterparty_account_number,
            counterparty_bank, counterparty_bank_country_code, counterparty_name, counterparty_inn,
            payment_purpose, terminal_id, location, operation_channel, document_number,
            exchange_rate, is_reversal, linked_operation_id, created_at, updated_at
        ])

# Создание DataFrame
columns = [
    'operation_id', 'client_id', 'account_id', 'account_number', 'operation_type', 'operation_date',
    'operation_amount', 'currency', 'transaction_fee', 'operation_status', 'counterparty_account_number',
    'counterparty_bank', 'counterparty_bank_country_code', 'counterparty_name', 'counterparty_inn',
    'payment_purpose', 'terminal_id', 'location', 'operation_channel', 'document_number',
    'exchange_rate', 'is_reversal', 'linked_operation_id', 'created_at', 'updated_at'
]

transaction = pd.DataFrame(data, columns=columns)

transaction.sample(10)

,operation_id,client_id,account_id,account_number,operation_type,operation_date,operation_amount,currency,transaction_fee,operation_status,...,payment_purpose,terminal_id,location,operation_channel,document_number,exchange_rate,is_reversal,linked_operation_id,created_at,updated_at
8079,708798_NZL,60178_UXL,791310_CRD,4070946486341457,ЗАЧИСЛЕНИЕ,2024-10-30 08:25:45,18696,USD,144.0,ЗАВЕРШЕНА,...,оплата по счету,53277.0,Республика Конго,БАНКОМАТ,278684,90,N,None,2024-10-30T13:25:45,2024-10-30T13:25:45
2503,600472_MUI,68703_VKH,370735_CRD,4070295269292516,ЗАЧИСЛЕНИЕ,2024-09-16 04:07:20,35288,RUB,220.0,ЗАВЕРШЕНА,...,оплата по контракту,NaN,NaN,МОБИЛЬНОЕ ПРИЛОЖЕНИЕ,5078822,1,N,None,2024-09-16T09:07:20,2024-09-16T09:07:20
1275,570510_KOK,95855_DLX,741138_CRD,4070956431910333,СПИСАНИЕ,2024-09-22 00:33:23,34910,USD,379.0,ЗАВЕРШЕНА,...,перевод средств в казино,NaN,NaN,МОБИЛЬНОЕ ПРИЛОЖЕНИЕ,980848,90,N,None,2024-09-22T05:33:23,2024-09-22T05:33:23
3152,287437_RXX,56079_YNX,922164_CRD,4070642286492767,ЗАЧИСЛЕНИЕ,2024-10-03 09:12:06,78876,RUB,NaN,ЗАВЕРШЕНА,...,оплата игры в рулетку,NaN,NaN,ИНТЕРНЕТ-БАНК,7705697,1,N,None,2024-10-03T14:12:06,2024-10-03T14:12:06
18502,601516_NIB,10050_MJK,679754_CRD,4070744230882029,СПИСАНИЕ,2024-11-02 17:10:13,42030,USD,NaN,ЗАВЕРШЕНА,...,перевод средств на банковский депозит,NaN,NaN,ИНТЕРНЕТ-БАНК,8442325,90,N,None,2024-11-02T22:10:13,2024-11-02T22:10:13
3544,965263_DNC,96795_XQH,252567_CRD,4070490111193817,ЗАЧИСЛЕНИЕ,2024-10-20 15:34:32,99189,RUB,123.0,ЗАВЕРШЕНА,...,погашение кредита,32399.0,к. Усть-Ишим,БАНКОМАТ,3131075,1,N,None,2024-10-20T20:34:32,2024-10-20T20:34:32
8142,572682_INW,99010_ALW,176069_CRD,4070960703309224,СПИСАНИЕ,2024-10-30 07:03:46,31310,RUB,NaN,ЗАВЕРШЕНА,...,перевод на счет,NaN,NaN,МОБИЛЬНОЕ ПРИЛОЖЕНИЕ,2672351,1,N,None,2024-10-30T12:03:46,2024-10-30T12:03:46
12076,668372_SSU,62906_SBE,555078_CRD,4070265786268427,ЗАЧИСЛЕНИЕ,2024-11-11 14:42:49,36852,EUR,293.0,ЗАВЕРШЕНА,...,игра в рулетку,68893.0,Алжир,БАНКОМАТ,1263622,100,N,None,2024-11-11T19:42:49,2024-11-11T19:42:49
5088,494765_LRC,89323_HTR,968094_CRD,4070633358533790,СПИСАНИЕ,2024-10-15 23:56:57,58681,USD,338.0,ЗАВЕРШЕНА,...,перевод на счет казино,NaN,NaN,МОБИЛЬНОЕ ПРИЛОЖЕНИЕ,7684350,90,N,None,2024-10-16T04:56:57,2024-10-16T04:56:57
7183,810515_QBQ,93756_DPU,527668_CRD,4070171135670273,СПИСАНИЕ,2024-10-11 19:04:15,19696,USD,NaN,ЗАВЕРШЕНА,...,перевод на счет казино,70857.0,Малави,БАНКОМАТ,5071475,90,N,None,2024-10-12T00:04:15,2024-10-12T00:04:15


In [ ]:
# Проверим временной отрезок
print(f"Минимальная дата: {transaction['operation_date'].min()} \nМаксимальная дата: {transaction['operation_date'].max()}")

Минимальная дата: 2024-09-13 00:00:16 
Максимальная дата: 2024-11-13 23:59:46


In [ ]:
# Распределение операций по статусам
transaction['operation_status'].value_counts()

,count
operation_status,
ЗАВЕРШЕНА,18021
ОЖИДАЕТСЯ,999
ОТКЛОНЕНА,980


## Экспорт файлов

In [ ]:
client_fl.to_csv(r'C:\datasets\client_fl.csv', index=False, encoding='cp1251')
client_ul.to_csv(r'C:\datasets\client_ul.csv', index=False, encoding='cp1251')
account.to_csv(r'C:\datasets\account.csv', index=False, encoding='cp1251')
transaction.to_csv(r'C:\datasets\transaction.csv', index=False, encoding='cp1251')